> ⚠️ **작업 중 (Work in Progress)**: 이 노트북은 현재 개발 중입니다. 일부 코드가 불완전하거나 변경될 수 있습니다.

## 📋 목차

- [환경 준비](#환경-준비)
- [Resource Group 생성](#resource-group-생성)
- [Foundry 리소스 생성](#foundry-리소스-생성)
- [리소스 확인](#리소스-확인)
- [다음 단계](#다음-단계)

## 🎯 학습 목표

- Azure CLI로 Resource Group 생성
- Azure CLI 또는 Bicep으로 Microsoft Foundry 리소스 생성
- 코드 기반 Infrastructure as Code (IaC) 실습

## ⏱️ 예상 소요 시간

약 10분

## 환경 준비

### 필수 요구사항

1. **Azure CLI 설치 확인**
   - 아래 셀을 실행하여 Azure CLI가 설치되어 있는지 확인합니다.
   - 설치되지 않은 경우: [Azure CLI 설치 가이드](https://learn.microsoft.com/cli/azure/install-azure-cli)

2. **Azure 로그인**
   - Azure CLI로 Azure 계정에 로그인합니다.

## Python 가상환경 설정

가상환경을 사용하면 프로젝트별로 독립적인 Python 패키지를 관리할 수 있습니다.

### 가상환경이란?

가상환경(Virtual Environment)은 프로젝트마다 독립적인 Python 실행 환경을 만들어줍니다.

**장점:**
- 프로젝트별로 다른 버전의 패키지 사용 가능
- 시스템 Python 환경을 깨끗하게 유지
- 팀원 간 동일한 개발 환경 공유 용이

### 가상환경 생성 및 활성화

**1. 터미널에서 가상환경 생성**

macOS / Linux:
```bash
python3 -m venv .venv
```

Windows:
```bash
python -m venv .venv
```

**2. 가상환경 활성화**

macOS / Linux:
```bash
source .venv/bin/activate
```

Windows (PowerShell):
```powershell
.venv\Scripts\Activate.ps1
```

Windows (CMD):
```cmd
.venv\Scripts\activate.bat
```

**3. 활성화 확인**

활성화되면 터미널 프롬프트 앞에 `(.venv)`가 표시됩니다:
```
(.venv) user@machine:~/project$
```

### VS Code Jupyter 사용 시

VS Code에서 Jupyter Notebook을 사용하는 경우:
1. 가상환경 생성 후
2. 우측 상단 **커널 선택** 버튼 클릭
3. **'.venv' 환경** 선택

이후 모든 셀은 자동으로 가상환경에서 실행됩니다.

In [ ]:
# 필수 패키지 설치
# 가상환경이 활성화된 상태에서 실행하세요

!pip install -q azure-ai-projects azure-identity azure-mgmt-resource

print("✅ 필수 패키지 설치 완료!")
print("\n📦 설치된 패키지:")
print("   - azure-ai-projects")
print("   - azure-identity")
print("   - azure-mgmt-resource")
print("\n💡 Azure CLI는 시스템 레벨에서 별도 설치가 필요합니다:")
print("   https://learn.microsoft.com/cli/azure/install-azure-cli")

In [ ]:
# PATH 환경변수 설정
# Jupyter 커널에서 Azure CLI를 찾을 수 있도록 경로 추가
import os
import subprocess

# Azure CLI가 설치될 수 있는 여러 경로 확인
possible_paths = [
    "/opt/homebrew/bin",  # macOS (Apple Silicon)
    "/usr/local/bin",     # macOS (Intel) / Linux
    "/usr/bin",           # Linux / GitHub Codespaces
    "/home/linuxbrew/.linuxbrew/bin"  # Linux Homebrew
]

# az 명령어 경로 찾기
az_path = None
try:
    result = subprocess.run(['which', 'az'], capture_output=True, text=True)
    if result.returncode == 0:
        az_path = os.path.dirname(result.stdout.strip())
        print(f"🔍 Azure CLI 발견: {result.stdout.strip()}")
except:
    pass

# 발견된 경로 또는 가능한 경로들을 PATH에 추가
paths_to_add = []
if az_path and az_path not in os.environ.get("PATH", ""):
    paths_to_add.append(az_path)
else:
    # which로 찾지 못한 경우 가능한 경로들 추가
    for path in possible_paths:
        if os.path.exists(path) and path not in os.environ.get("PATH", ""):
            paths_to_add.append(path)

if paths_to_add:
    new_path = ":".join(paths_to_add) + ":" + os.environ.get("PATH", "")
    os.environ["PATH"] = new_path
    print(f"✅ PATH에 추가된 경로: {', '.join(paths_to_add)}")
else:
    print("✅ PATH가 이미 올바르게 설정되어 있습니다.")

print(f"\n💡 현재 PATH (처음 150자): {os.environ['PATH'][:150]}...")

In [ ]:
# Azure 인증 (Python SDK 사용)
from azure.identity import InteractiveBrowserCredential, DeviceCodeCredential
from azure.mgmt.resource import SubscriptionClient
import os

# 테넌트 ID 설정 (필요시 변경)
TENANT_ID = os.getenv("AZURE_TENANT_ID", "16b3c013-d300-468d-ac64-7eda0820b6d3")

# GitHub Codespaces 또는 원격 환경 감지
IS_CODESPACES = os.getenv("CODESPACES") == "true" or os.getenv("CODESPACE_NAME") is not None
IS_REMOTE = os.getenv("REMOTE_CONTAINERS") == "true" or IS_CODESPACES

print("🔐 Azure 인증 시작...")
print(f"Tenant ID: {TENANT_ID}")

if IS_REMOTE:
    print("\n💡 원격 환경(Codespaces/Remote Container)이 감지되었습니다.")
    print("디바이스 코드 인증을 사용합니다.\n")
else:
    print("브라우저가 열리면 Azure 계정으로 로그인하세요.\n")

try:
    # 환경에 따라 다른 인증 방식 사용
    if IS_REMOTE:
        # Codespaces/Remote: DeviceCodeCredential 사용
        credential = DeviceCodeCredential(tenant_id=TENANT_ID)
        print("📱 아래 단계를 따라 인증하세요:")
        print("   1. 아래 URL을 로컬 브라우저에서 엽니다")
        print("   2. 표시되는 코드를 입력합니다")
        print("   3. Azure 계정으로 로그인합니다\n")
    else:
        # 로컬: InteractiveBrowserCredential 사용
        credential = InteractiveBrowserCredential(tenant_id=TENANT_ID)
    
    # 구독 목록 조회
    subscription_client = SubscriptionClient(credential)
    subscriptions = list(subscription_client.subscriptions.list())
    
    print("✅ Azure 인증 완료!")
    print("\n" + "=" * 80)
    print("📋 사용 가능한 구독 목록:")
    print("=" * 80)
    
    for i, sub in enumerate(subscriptions, 1):
        print(f"\n{i}. {sub.display_name}")
        print(f"   Subscription ID: {sub.subscription_id}")
        print(f"   상태: {sub.state}")
    
    print("\n" + "=" * 80)
    print(f"✅ 총 {len(subscriptions)}개의 구독을 찾았습니다.")
    
    # 기본 구독 정보 저장 (첫 번째 구독)
    if subscriptions:
        default_sub = subscriptions[0]
        print(f"\n💡 기본 구독: {default_sub.display_name}")
        print(f"   Subscription ID: {default_sub.subscription_id}")
        
        # 환경 변수로 저장
        os.environ["AZURE_SUBSCRIPTION_ID"] = default_sub.subscription_id
        print("\n✅ 기본 구독 ID가 환경 변수에 저장되었습니다.")
    
except Exception as e:
    print(f"\n⚠️ 인증 실패: {e}")
    print("\n💡 해결 방법:")
    print("   1. 테넌트 ID가 올바른지 확인")
    if IS_REMOTE:
        print("   2. 디바이스 코드 인증 URL을 로컬 브라우저에서 열었는지 확인")
        print("   3. 표시된 코드를 정확히 입력했는지 확인")
    else:
        print("   2. 브라우저에서 로그인 완료 확인")
    print("   4. 계정에 해당 테넌트 접근 권한이 있는지 확인")

## Resource Group 생성

Resource Group은 Azure 리소스를 논리적으로 그룹화하는 컨테이너입니다.

In [ ]:
# Resource Group 생성
!az group create \
    --name foundry-code \
    --location swedencentral

# 생성 확인
!az group show \
    --name foundry-code \
    --output table

## Foundry 리소스 생성

고유한 이름으로 Foundry(AIServices) 리소스를 생성합니다.

In [ ]:
# 환경 변수 설정
import os
import random
import string

# 고유한 이름 생성
random_suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=6))
FOUNDRY_NAME = f"foundry-{random_suffix}"
PROJECT_NAME = "default-project"
RESOURCE_GROUP = "foundry-code"
LOCATION = "swedencentral"

print(f"✅ 환경 변수 설정")
print(f"   Foundry: {FOUNDRY_NAME}")
print(f"   Project: {PROJECT_NAME}")
print(f"   Resource Group: {RESOURCE_GROUP}")
print(f"   Location: {LOCATION}")

In [ ]:
# Step 1: Foundry Resource 생성 (AIServices)
import subprocess
import json

subscription_id = os.environ.get('AZURE_SUBSCRIPTION_ID', '')

# 🔧 수정: API 버전을 2025-04-01-preview로 변경 (allowProjectManagement 지원)
foundry_url = f"https://management.azure.com/subscriptions/{subscription_id}/resourceGroups/{RESOURCE_GROUP}/providers/Microsoft.CognitiveServices/accounts/{FOUNDRY_NAME}?api-version=2025-04-01-preview"

foundry_body = json.dumps({
    "location": LOCATION,
    "kind": "AIServices",
    "sku": {"name": "S0"},
    "identity": {"type": "SystemAssigned"},
    "properties": {
        "customSubDomainName": FOUNDRY_NAME,
        "publicNetworkAccess": "Enabled",
        "allowProjectManagement": True
    }
})

print(f"📌 Step 1: Foundry Resource 생성 중: {FOUNDRY_NAME}")
print("   (AIServices 타입, API v2025-04-01-preview)")
print("   ✅ allowProjectManagement: True")
print("   💡 disableLocalAuth: 설정하지 않음 (Azure 기본값 적용)")

result = subprocess.run(
    ['az', 'rest', '--method', 'PUT', '--url', foundry_url, '--body', foundry_body],
    capture_output=True, text=True
)

if result.returncode == 0:
    print("✅ Foundry Resource 생성 완료")
    foundry_info = json.loads(result.stdout)
    foundry_id = foundry_info.get('id', '')
    os.environ["FOUNDRY_ID"] = foundry_id
    
    # allowProjectManagement 확인
    properties = foundry_info.get('properties', {})
    allow_project = properties.get('allowProjectManagement')
    disable_local_auth = properties.get('disableLocalAuth')
    
    print(f"   Foundry ID: {foundry_id[:70]}...")
    print(f"   allowProjectManagement: {allow_project}")
    print(f"   disableLocalAuth: {disable_local_auth}")
    
    if allow_project and not disable_local_auth:
        print("   ✅ Project 생성 및 API Key 인증 준비 완료")
    else:
        print(f"   ⚠️ 설정 확인 필요")
else:
    print(f"⚠️ Foundry 생성 실패: {result.stderr}")

In [ ]:
# Step 2: Foundry Project 생성 (Subresource)
import subprocess
import json
import time

print(f"📌 Step 2: Foundry Project 생성 중: {PROJECT_NAME}")

foundry_id = os.environ.get('FOUNDRY_ID', '')
if foundry_id:
    # Foundry Resource 생성 완료 대기
    time.sleep(5)
    
    # Project를 Foundry의 subresource로 생성
    project_url = f"https://management.azure.com{foundry_id}/projects/{PROJECT_NAME}?api-version=2025-04-01-preview"
    project_body = json.dumps({
        "location": LOCATION,
        "identity": {"type": "SystemAssigned"},
        "properties": {
            "friendlyName": PROJECT_NAME,
            "description": f"Foundry Project: {PROJECT_NAME}"
            # disableLocalAuth 설정 제거 - Azure 기본값 확인
        }
    })
    
    print("   💡 API Key 인증: 설정하지 않음 (Azure 기본값 적용)")
    
    result = subprocess.run(
        ['az', 'rest', '--method', 'PUT', '--url', project_url, '--body', project_body],
        capture_output=True, text=True
    )
    
    if result.returncode == 0:
        print("✅ Foundry Project 생성 완료")
        project_info = json.loads(result.stdout)
        project_id = project_info.get('id', '')
        os.environ["PROJECT_ID"] = project_id
        print(f"   Project ID: {project_id[:70]}...")
        
        # disableLocalAuth 확인
        properties = project_info.get('properties', {})
        disable_local_auth = properties.get('disableLocalAuth', True)
        if not disable_local_auth:
            print("   ✅ API Key 인증이 활성화되었습니다")
        else:
            print("   ⚠️ API Key 인증이 아직 비활성화 상태입니다")
    else:
        print(f"⚠️ Project 생성 실패: {result.stderr}")
else:
    print("⚠️ FOUNDRY_ID가 없습니다. 이전 셀을 먼저 실행하세요.")

In [ ]:
# Foundry Resource 및 Project 확인
import subprocess

print("📋 생성된 리소스 확인:\n")

result = subprocess.run(
    ['az', 'cognitiveservices', 'account', 'show',
     '--name', FOUNDRY_NAME,
     '--resource-group', RESOURCE_GROUP,
     '--query', '{Name:name, Kind:kind, Location:location, Endpoint:properties.endpoint}',
     '--output', 'table'],
    capture_output=True, text=True
)

if result.returncode == 0:
    print("Foundry Resource:")
    print(result.stdout)
    print(f"\n✅ Foundry Resource와 Project가 생성되었습니다!")
    print(f"💡 Portal: https://ai.azure.com")
else:
    print(f"⚠️ 확인 실패: {result.stderr}")

## API Key 및 Endpoint 조회

Foundry 리소스의 API Key와 Endpoint를 조회합니다.

In [ ]:
# API Key 생성 (Foundry Resource용)
import subprocess
import json

print(f"📌 Foundry API Key 생성")
print("💡 Foundry Project는 부모 AIServices 리소스의 Key를 사용합니다.\n")

foundry_id = os.environ.get('FOUNDRY_ID', '')
if foundry_id:
    # 방법 1: Azure CLI 명령어로 Key 가져오기
    print("🔑 방법 1: Azure CLI로 Key 조회")
    result = subprocess.run(
        ['az', 'cognitiveservices', 'account', 'keys', 'list',
         '--name', FOUNDRY_NAME,
         '--resource-group', RESOURCE_GROUP],
        capture_output=True, text=True
    )
    
    if result.returncode == 0:
        keys = json.loads(result.stdout)
        primary_key = keys.get('key1', '')
        
        if primary_key:
            print("✅ API Key 조회 완료")
            print(f"   Key1: {primary_key[:20]}...")
            os.environ["FOUNDRY_API_KEY"] = primary_key
            
            # Endpoint도 가져오기
            endpoint_result = subprocess.run(
                ['az', 'cognitiveservices', 'account', 'show',
                 '--name', FOUNDRY_NAME,
                 '--resource-group', RESOURCE_GROUP,
                 '--query', 'properties.endpoint',
                 '--output', 'tsv'],
                capture_output=True, text=True
            )
            
            if endpoint_result.returncode == 0:
                base_endpoint = endpoint_result.stdout.strip()
                # Project 엔드포인트 구성
                # cognitiveservices.azure.com을 services.ai.azure.com으로 변경
                project_endpoint = base_endpoint.replace(
                    ".cognitiveservices.azure.com/",
                    ".services.ai.azure.com/"
                ).rstrip('/') + f"/api/projects/{PROJECT_NAME}"
                
                os.environ["FOUNDRY_ENDPOINT"] = project_endpoint
                print(f"   Base Endpoint: {base_endpoint}")
                print(f"   Project Endpoint: {project_endpoint}")
        else:
            print("⚠️ Key를 찾을 수 없습니다.")
    else:
        print(f"⚠️ Key 조회 실패: {result.stderr}")
        print("\n🔑 방법 2: REST API로 Key 조회 시도")
        
        # 방법 2: REST API로 시도
        key_url = f"https://management.azure.com{foundry_id}/listKeys?api-version=2025-04-01-preview"
        
        result2 = subprocess.run(
            ['az', 'rest', '--method', 'POST', '--url', key_url],
            capture_output=True, text=True
        )
        
        if result2.returncode == 0:
            keys = json.loads(result2.stdout)
            primary_key = keys.get('key1', '') or keys.get('primaryKey', '')
            
            if primary_key:
                print("✅ API Key 조회 완료 (REST API)")
                print(f"   Key: {primary_key[:20]}...")
                os.environ["FOUNDRY_API_KEY"] = primary_key
            else:
                print("⚠️ Key를 찾을 수 없습니다.")
        else:
            print(f"⚠️ REST API도 실패: {result2.stderr}")
else:
    print("⚠️ FOUNDRY_ID가 없습니다.")

print(f"\n💡 Portal에서 확인: https://ai.azure.com")
print(f"💡 또는 Azure Portal에서 '{FOUNDRY_NAME}' 리소스의 Keys and Endpoint 메뉴 확인")


### Azure Portal에서 확인 (선택사항)

생성된 리소스를 시각적으로 확인하려면:

1. [Azure Portal](https://portal.azure.com)에 접속
2. Resource Group `foundry` 검색
3. Foundry 리소스 클릭하여 상세 정보 확인

또는 [Microsoft Foundry Portal](https://ai.azure.com)에서 프로젝트를 확인할 수 있습니다.

## 환경 변수 저장

다음 노트북에서 사용할 주요 변수들을 파일로 저장합니다.

In [ ]:
# 환경 변수를 JSON 파일로 저장
import json

config = {
    "FOUNDRY_NAME": FOUNDRY_NAME,
    "PROJECT_NAME": PROJECT_NAME,
    "RESOURCE_GROUP": RESOURCE_GROUP,
    "LOCATION": LOCATION,
    "AZURE_SUBSCRIPTION_ID": os.environ.get("AZURE_SUBSCRIPTION_ID", ""),
    "TENANT_ID": TENANT_ID,
    "FOUNDRY_ID": os.environ.get("FOUNDRY_ID", ""),
    "PROJECT_ID": os.environ.get("PROJECT_ID", ""),
    "FOUNDRY_API_KEY": os.environ.get("FOUNDRY_API_KEY", ""),
    "FOUNDRY_ENDPOINT": os.environ.get("FOUNDRY_ENDPOINT", "")
}

config_file = ".foundry_config.json"
with open(config_file, 'w') as f:
    json.dump(config, f, indent=2)

print(f"✅ 설정 파일 저장: {config_file}")
print(f"\n📋 저장된 정보:")
print(f"   Foundry: {FOUNDRY_NAME}")
print(f"   Project: {PROJECT_NAME}")
print(f"   Location: {LOCATION}")
print(f"   Endpoint: {os.environ.get('FOUNDRY_ENDPOINT', 'N/A')}")
print(f"   API Key: {'설정됨' if os.environ.get('FOUNDRY_API_KEY') else '미설정'}")
print(f"   Type: Foundry Project (AIServices + Project subresource)")
print(f"\n💡 다음 노트북에서 이 설정을 자동으로 로드합니다.")


## 📚 추가 리소스

- [Microsoft Foundry 문서](https://learn.microsoft.com/en-us/azure/ai-foundry/what-is-azure-ai-foundry?view=foundry)
- [Azure Resource Manager 개요](https://learn.microsoft.com/azure/azure-resource-manager/management/overview)
- [Azure 리전 및 가용성 영역](https://learn.microsoft.com/azure/reliability/availability-zones-overview)

## 다음 단계

환경 설정이 완료되었습니다! 이제 다음 모듈로 진행하세요:

➡️ **[02. 모델 및 배포](./02-models.ipynb)**: 다양한 AI 모델을 탐색하고 배포하는 방법을 학습합니다.